In [3]:
import cv2
import glob
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.utils import to_categorical

# Caminhos para os dados de treino
#path_normal_train = "C:/Users/Artur Vinicius/Documents/Python/chest_xray/train/NORMAL/*.jpeg"
#path_pneumonia_train = "C:/Users/Artur Vinicius/Documents/Python/chest_xray/train/PNEUMONIA/*.jpeg"
path_total = "C:/Users/Artur Vinicius/Documents/Python/chest_xray_v4/total_data/*.jpeg"
# Caminhos para os dados de teste
#path_normal_test = "C:/Users/Artur Vinicius/Documents/Python/chest_xray/test/NORMAL/*.jpeg"
#path_pneumonia_test = "C:/Users/Artur Vinicius/Documents/Python/chest_xray/test/PNEUMONIA/*.jpeg"

# Dimensões desejadas para as imagens
width = 128
height = 128
dim = (width, height)

def load_image(path):
    images = []
    for file in glob.glob(path):
        image = cv2.imread(file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        resized_image = cv2.resize(image, dim)
        images.append(resized_image)
    return images


In [4]:
x_data = load_image(path_total)

In [5]:
print(type(x_data))

<class 'list'>


In [6]:
for i in range (len(x_data)):
    print('New Dimensions : ',x_data[i].shape)

New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions :  (128, 128)
New Dimensions

In [7]:
y_data = [] # Target
string_bacteria = "bacteria"
string_virus = "virus"

for file_path in glob.glob(path_total):
    file_name = file_path.split("/")[-1]  # Extrair apenas o nome do arquivo
    if string_bacteria in file_name:
        y_data.append(1) # Rotular os RX com Pneumonia Bacteriana com 1
    elif string_virus in file_name:
        y_data.append(2) # Rotular os RX com Pneumonia Viral com 2
    elif (string_bacteria not in file_name) and (string_virus not in file_name):
        y_data.append(0) # Rotular os RX sem Pneumonia com o valor 0

In [ ]:
print(type(y_data))

In [8]:
x_data = np.asarray(x_data).astype(np.float32)
y_data = np.asarray(y_data).astype(np.float32)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.4, random_state=42)

In [10]:
# Definir o modelo
from keras.layers import MaxPooling2D
model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=(128,128,1)))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Treinar o modelo
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1)

Epoch 1/5
28/28 [==============================] - 28s 982ms/step - loss: 1286.4600 - accuracy: 0.4885
Epoch 2/5
28/28 [==============================] - 30s 1s/step - loss: 0.6752 - accuracy: 0.7498
Epoch 3/5
28/28 [==============================] - 30s 1s/step - loss: 0.4396 - accuracy: 0.8332
Epoch 4/5
28/28 [==============================] - 29s 1s/step - loss: 0.2865 - accuracy: 0.8984
Epoch 5/5
23/28 [=======================>......] - ETA: 5s - loss: 0.1668 - accuracy: 0.9429

In [ ]:
#y_test_categorical = to_categorical(y_test)

y_pred = model.predict(X_test)

In [ ]:
print(y_pred)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)